In [ ]:
# https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/langchain_bigquery_data_loader.ipynb
# https://github.com/GoogleCloudPlatform/data-beans/blob/main/colab-enterprise/gen-ai-demo/Common-Themes-RAG.ipynb
# https://cloud.google.com/bigquery/docs/vector-index-text-search-tutorial#terraform
# https://medium.com/@dhafnar/building-a-serverless-rag-system-with-bigquery-and-gemini-12e66328f622

In [ ]:
#1. CONNECT TO BIGQUERY: 
# create a Cloud resource connection and get the connection's service account
# create the connection in the same location as the dataset you created in the previous step


#2. CREATE THE REMOTE MODEL FOR TEXT EMBEDDING GENERATION:
# CREATE OR REPLACE MODEL `bqml_tutorial.embedding_model`
#   REMOTE WITH CONNECTION `LOCATION.CONNECTION_ID`
#   OPTIONS (ENDPOINT = 'text-embedding-004');


#3. GENERATE TEXT EMBEDDINGS:
# generate text embeddings from patent abstracts using the ML.GENERATE_EMBEDDING function, 
# and then write them to a BigQuery table so that they can be searched.
# CREATE OR REPLACE TABLE `bqml_tutorial.embeddings` AS
# SELECT * FROM ML.GENERATE_EMBEDDING(
#   MODEL `bqml_tutorial.embedding_model`,
#   (
#     SELECT *, abstract AS content
#     FROM `patents-public-data.google_patents_research.publications`
#     WHERE LENGTH(abstract) > 0 AND LENGTH(title) > 0 AND country = 'Singapore'
#   )
# )
# WHERE LENGTH(ml_generate_embedding_status) = 0;


#4. CREATE A VECTOR INDEX:
# to create a vector index, use the CREATE VECTOR INDEX data definition language (DDL)
# CREATE OR REPLACE VECTOR INDEX my_index
# ON `bqml_tutorial.embeddings`(ml_generate_embedding_result)
# OPTIONS(index_type = 'IVF',
#   distance_type = 'COSINE',
#   ivf_options = '{"num_lists":500}')


#5. VERIFY VECTOR INDEX CREATION:
# check whether the index is ready to be used by querying the INFORMATION_SCHEMA.VECTOR_INDEXES view 
# and verifying that the coverage_percentage column value is greater than 0 
# and the last_refresh_time column value isn't NULL
# SELECT table_name, index_name, index_status,
# coverage_percentage, last_refresh_time, disable_reason
# FROM `PROJECT_ID.bqml_tutorial.INFORMATION_SCHEMA.VECTOR_INDEXES`


#6. PERFORM A TEXT SIMILARITY SEARCH USING THE VECTOR INDEX:
# use the VECTOR_SEARCH function to search for the top 5 relevant patents that match embeddings generated from a text query
# the model you use to generate the embeddings in this query must be the same as the one you use to generate the embeddings 
# in the table you are comparing against, otherwise the search results won't be accurate
# SELECT query.query, base.publication_number, base.title, base.abstract
# FROM VECTOR_SEARCH(
#   TABLE `bqml_tutorial.embeddings`, 'ml_generate_embedding_result',
#   (
#   SELECT ml_generate_embedding_result, content AS query
#   FROM ML.GENERATE_EMBEDDING(
#   MODEL `bqml_tutorial.embedding_model`,
#   (SELECT 'improving password security' AS content))
#   ),
#   top_k => 5, options => '{"fraction_lists_to_search": 0.01}')


#7. CREATE THE REMOTE MODEL FOR TEXT GENERATION:
# create a remote model that represents a hosted Vertex AI text generation model
# CREATE OR REPLACE MODEL `bqml_tutorial.text_model`
#   REMOTE WITH CONNECTION `LOCATION.CONNECTION_ID`
#   OPTIONS (ENDPOINT = 'text-bison-32k');


#8. GENERATE TEXT AUGMENTED BY VECTOR SEARCH RESULTS
# feed the search results as prompts to generate text with the ML.GENERATE_TEXT function
# SELECT ml_generate_text_llm_result AS generated, prompt
# FROM ML.GENERATE_TEXT(
#   MODEL `bqml_tutorial.text_model`,
#   (
#     SELECT CONCAT(
#       'Propose some project ideas to improve user password security using the context below: ',
#       STRING_AGG(
#         FORMAT("patent title: %s, patent abstract: %s", base.title, base.abstract),
#         ',\n')
#       ) AS prompt,
#     FROM VECTOR_SEARCH(
#       TABLE `bqml_tutorial.embeddings`, 'ml_generate_embedding_result',
#       (
#         SELECT ml_generate_embedding_result, content AS query
#         FROM ML.GENERATE_EMBEDDING(
#           MODEL `bqml_tutorial.embedding_model`,
#          (SELECT 'improving password security' AS content)
#         )
#       ),
#     top_k => 5, options => '{"fraction_lists_to_search": 0.01}')
#   ),
#   STRUCT(600 AS max_output_tokens, TRUE AS flatten_json_output));
